In [4]:
"""This file :
                1. Scrapes names and relevant links to player wiki from url below and stores this data in Names.xlsx file or df 
                    Dataframe.
                2. Scrape parsed wikipage files for birthday and play-hand information."""

#Import packages
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import requests
#Set fixed values
url="https://en.wikipedia.org/wiki/List_of_male_singles_tennis_players" # wiki url with list of all players
names_to_delete = ["Jim Osborne"] #To be removed from analysis(instead of writing exception later)
###1.Obtain list of all male tennis players and their wiki link
response = requests.get(url)
soup =  BeautifulSoup(response.text, 'html.parser')
a = soup.find('table',{'class':"wikitable"})
vikilist = []
namelist = []
index_to_delete = []
for tr in a.find_all(class_='vcard'):
    x = tr.find("a")
    namelist.append(x.string)
    vikilist.append(x["href"])
#Remove names that are too old fashioned etc.
for name in names_to_delete:
    x = namelist.index(name)
    index_to_delete.append(x)
for x in index_to_delete:
    del namelist[x]
    del vikilist[x]
#Create DataFrame
d = {'Name':namelist,'Viki':vikilist}
df = pd.DataFrame(d)
df.to_excel('Names.xlsx')

,Name,Viki
0,Robert Abdesselam,/wiki/Robert_Abdesselam
1,José Acasuso,/wiki/Jos%C3%A9_Acasuso
2,András Ádám-Stolpa,/wiki/Andr%C3%A1s_%C3%81d%C3%A1m-Stolpa
3,Andre Agassi,/wiki/Andre_Agassi
4,Ronald Agénor,/wiki/Ronald_Ag%C3%A9nor
...,...,...
623,Mariano Zabaleta,/wiki/Mariano_Zabaleta
624,Vladimír Zedník,/wiki/Vladim%C3%ADr_Zedn%C3%ADk
625,Slobodan Živojinović,/wiki/Slobodan_%C5%BDivojinovi%C4%87
626,Alexander Zverev,/wiki/Alexander_Zverev


In [6]:
%%time
#Create BeautifulSoup object out of wiki  page for each player, then parse it (around 15min runtime,629 player names)
j=0
players = BeautifulSoup()
for i in vikilist:
        url="https://en.wikipedia.org" + i
        response = requests.get(url)
        players[j] =  BeautifulSoup(response.text, 'html.parser')
        j+=1
#628+1

Wall time: 2min 28s


In [12]:
#2. Using BeautifulSoup object created above, pick up date of birth and information on which hand the player plays
data = []
for i in range(len(vikilist)):
    try:
        x = players[i].find('span',class_="bday").text
    except AttributeError:
        x = players[i].find('th',string="Born").next_sibling.text
    except:
        x = "Sth wrong here chief"
    try:
        z = players[i].find('th',string="Plays").next_sibling.text
        
    except AttributeError:
        z = "Data not available"
    finally:
        y=i,x,z
        data.append(y)

In [8]:
data

[(0, '1920-01-27', 'Right-handed'),
 (1, '1982-10-20', 'Right-handed (one-handed backhand)'),
 (2, '1921-09-15', 'Data not available'),
 (3, '1970-04-29', 'Right-handed (two-handed backhand)'),
 (4, '1964-11-13', 'Right-handed (one-handed backhand)'),
 (5, '1962-03-22', 'Right-handed (one-handed backhand)'),
 (6, '1988-04-14', 'Right-handed (two-handed backhand)'),
 (7, '1973-05-24', 'Right-handed (one-handed backhand)'),
 (8, '1880-08-14', 'Right-handed (one-handed backhand)'),
 (9, '1951-07-04', 'Right-handed (one-handed backhand)'),
 (10, '1904-12-08', 'Right-handed (1-handed backhand)'),
 (11, '1985-08-21', 'Right-handed (one-handed backhand)'),
 (12, '1895-11-12', 'right-handed (one-handed backhand)'),
 (13, '1920-10-27', 'Data not available'),
 (14, '1953-12-14', 'Right-handed (one-handed backhand)'),
 (15, '1984-03-30', 'Right-handed (two-handed backhand)'),
 (16, '1894-09-17', 'Right-handed (one-handed backhand)'),
 (17, '1986-05-18', 'Right-handed (two-handed backhand)'),
 (18

In [43]:
dataframe = pd.DataFrame (data, columns = ['index','birthday','plays'])
dataframe['name']=df['Name']
#Drop players with long date of birth as they are usually too old to be included anyway
for index, row in dataframe.iterrows():
    if len(row['birthday']) >10:
        dataframe.drop(index,inplace=True)
dataframe['birthday']= pd.to_datetime(dataframe['birthday'])
dataframe

,index,birthday,plays,name
0,0,1920-01-27,Right-handed,Robert Abdesselam
1,1,1982-10-20,Right-handed (one-handed backhand),José Acasuso
2,2,1921-09-15,Data not available,András Ádám-Stolpa
3,3,1970-04-29,Right-handed (two-handed backhand),Andre Agassi
4,4,1964-11-13,Right-handed (one-handed backhand),Ronald Agénor
...,...,...,...,...
623,623,1978-02-28,Right-handed (single-handed backhand),Mariano Zabaleta
624,624,1947-02-01,Data not available,Vladimír Zedník
625,625,1963-07-23,Right-handed (one-handed backhand),Slobodan Živojinović
626,626,1997-04-20,Right-handed (two-handed backhand),Alexander Zverev


In [47]:
dataframe[(dataframe['birthday'] > '1980-07-23')]

,index,birthday,plays,name
1,1,1982-10-20,Right-handed (one-handed backhand),José Acasuso
6,6,1988-04-14,Right-handed (two-handed backhand),Roberto Bautista Agut
11,11,1985-08-21,Right-handed (one-handed backhand),Nicolás Almagro
15,15,1984-03-30,Right-handed (two-handed backhand),Mario Ančić
17,17,1986-05-18,Right-handed (two-handed backhand),Kevin Anderson
...,...,...,...,...
597,597,1981-09-05,Right-handed (one-handed backhand),Filippo Volandri
605,605,1985-03-28,Right-handed (one-handed backhand),Stan Wawrinka
621,621,1982-06-25,Right-handed (one-handed backhand),Mikhail Youzhny
626,626,1997-04-20,Right-handed (two-handed backhand),Alexander Zverev


In [40]:
dataframe.dtypes

index                int64
birthday    datetime64[ns]
plays               object
name                object
dtype: object